<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Francisca Ruiz
- Nombre de alumno 2: Valentina Zuñiga


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install xgboost

In [31]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.6 MB/s eta 0:00:00


# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
!wget 'https://docs.google.com/uc?export=download&id=1o1u2K9nYnRB6wvVpYfTckgD24-CXCUiq' -O sales.csv

--2024-10-25 00:57:31--  https://docs.google.com/uc?export=download&id=1o1u2K9nYnRB6wvVpYfTckgD24-CXCUiq
Resolving docs.google.com (docs.google.com)... 74.125.201.113, 74.125.201.138, 74.125.201.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.201.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1o1u2K9nYnRB6wvVpYfTckgD24-CXCUiq&export=download [following]
--2024-10-25 00:57:31--  https://drive.usercontent.google.com/download?id=1o1u2K9nYnRB6wvVpYfTckgD24-CXCUiq&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.69.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 656341 (641K) [application/octet-stream]
Saving to: ‘sales.csv’

sales.csv           100%[===================>] 640.96K  --.-KB/s    in 0.01s 

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [4]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
set_config(transform_output="pandas")

# Separamos datos
X = df.drop('quantity', axis=1)
y = df['quantity']

# Convertimos columna date en tipo datetime
X['date'] = pd.to_datetime(X['date'])

# 1. Separar datos en conjunto de train (70%), val (20%) y test (10%)
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size=0.3333, random_state=42)

print(f'Cantidad datos de entrenamiento: {X_train.shape[0]}')
print(f'Cantidad datos de validación: {X_val.shape[0]}')
print(f'Cantidad datos de test: {X_test.shape[0]}')

Cantidad datos de entrenamiento: 5219
Cantidad datos de validación: 1491
Cantidad datos de test: 746


<ipython-input-4-c05c080fdbc6>:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X['date'] = pd.to_datetime(X['date'])


In [5]:
# 2. Se implementa FuncionTransformer para extraer dia, mes y año de la variable date
from sklearn.preprocessing import FunctionTransformer

def extract_date(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df

date_transformer = FunctionTransformer(extract_date)

In [23]:
# 3. Implemente un ColumnTransformer para procesar de manera adecuada los datos numéricos y categóricos.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Dividimos por categoria
categorical_data = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']
numerical_data = ['lat', 'long', 'pop', 'price']
date_data = ['date']
# Generamos col_transformer
col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_data),
        ('categorical', OneHotEncoder(sparse_output=False), categorical_data)
    ])

col_transformer.set_output(transform='pandas')

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['lat', 'long', 'pop', 'price']),
                                ('categorical',
                                 OneHotEncoder(sparse_output=False),
                                 ['city', 'shop', 'brand', 'container',
                                  'capacity', 'day', 'month', 'year'])])

In [24]:
# 4. Guarde los pasos anteriores en un Pipeline, dejando como último paso el regresor DummyRegressor para generar predicciones en base a promedios
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

pipeline_dummy = Pipeline([
    ('date_transformer', date_transformer),
    ('col_transformer', col_transformer),
    ('regressor', DummyRegressor(strategy="mean"))
])

In [25]:
# 5. Entrene el pipeline anterior y reporte la métrica mean_absolute_error sobre los datos de validación
from sklearn.metrics import mean_absolute_error

# se entrena
pipeline_dummy.fit(X_train, y_train)

# realizamos predicciones
y_pred = pipeline_dummy.predict(X_val)

# se calcula error
mae_dummy = mean_absolute_error(y_val, y_pred)
print(f'MAE Dummy: {mae_dummy}')

MAE Dummy: 13298.497767341096


Se obtiene un MAE de 13298, este valor se interpreta como la precisión del modelo, si este valor es menor entonces las predicciones se van acercando a los valores reales.

In [26]:
# 6. Finalmente, vuelva a entrenar el Pipeline pero esta vez usando XGBRegressor como modelo utilizando los parámetros por default.
from xgboost import XGBRegressor

pipeline_xgb = Pipeline([
    ('date_transformer', date_transformer),
    ('col_transformer', col_transformer),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(X_train, y_train)

y_pred = pipeline_xgb.predict(X_val)

mae_xgb = mean_absolute_error(y_val, y_pred)
print(f'MAE XGB: {mae_xgb}')

MAE XGB: 2433.320936196607


Podemos apreciar que el MAE disminuye considerablemente, resultando casi la mitad del MAE del DummyRegressor, por lo que el valor del XGB es un valor más preciso.

In [27]:
# 7. Guarde ambos modelos en un archivo .pkl
import pickle

with open('pipeline_dummy.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)

with open('pipeline_xgb.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [38]:
# 1. Se vuelve a entrenar pipeline

# Obtenemos nombres de las de las columnas del paso anterior
feature_names = col_transformer.get_feature_names_out()

# Se definen la motonicidad (0 para cualquier variable excepto price)
monotone_features = {name: 0 for name in feature_names}
monotone_features['num__price'] = -1

# Se genera pipeline
pipeline_monotono = Pipeline([
    ('date_transformer', date_transformer),
    ('col_transformer', col_transformer),
    ('regressor', XGBRegressor(monotone_constraints=tuple(monotone_features.values())))
])

pipeline_monotono.fit(X_train, y_train)

y_pred = pipeline_monotono.predict(X_val)

# 2. Reportamos MAE sobre conjunto de validación
mae_xgb_monotone = mean_absolute_error(y_val, y_pred)
print(f'MAE XGB monotono: {mae_xgb_monotone}')

MAE XGB monotono: 2485.2666868904466


3. El error disminuye levemente en comparación con el caso anterior, por lo tanto, no necesariamente el amigo tiene razón ya que si bien hay una mejora, esta no es tan significativa.

In [43]:
# 4. Guardamos modelos .pkl
with open('pipeline_xgb_monotonic.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [47]:
import optuna
from optuna.samplers import TPESampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Separamos datos
X = df.drop('quantity', axis=1)
y = df['quantity']

# Convertimos columna date en tipo datetime
X['date'] = pd.to_datetime(X['date'])
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size=0.3333, random_state=42)

# 1. Se implementa función
def objective(trial):
    # Se determina frecuencia minima para OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Generamos col transformer
    col_transformer = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_data),
            ('categorical', OneHotEncoder(min_frequency=min_frequency, sparse_output=False), categorical_data)
        ])
    col_transformer.set_output(transform='pandas')

    # Generamos diccionario con variable a optimizar
    params_xgb = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
              'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
              'max_depth': trial.suggest_int('max_depth', 3, 10),
              'max_leaves': trial.suggest_int('max_leaves', 0, 100),
              'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
              'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
              'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
              }

    # Se genera pipeline
    pipeline = Pipeline([
        ('date_transformer', date_transformer),
        ('col_transformer', col_transformer),
        ('regressor', XGBRegressor(
            learning_rate = params_xgb['learning_rate'],
            n_estimators = params_xgb['n_estimators'],
            max_depth = params_xgb['max_depth'],
            max_leaves = params_xgb['max_leaves'],
            min_child_weight = params_xgb['min_child_weight'],
            reg_alpha = params_xgb['reg_alpha'],
            reg_lambda = params_xgb['reg_lambda']
        ))
    ])

    # Se entrena, se predice y se calcula MAE
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    mae_xgb = mean_absolute_error(y_val, y_pred)

    # Almacenamos mejor pipeline
    trial.set_user_attr("pipeline_optuna", pipeline)

    return mae_xgb

<ipython-input-47-83ec57602e30>:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X['date'] = pd.to_datetime(X['date'])


In [48]:
# 2. Fijar el tiempo de entrenamiento a 5 minutos
time_train = 5*60

In [49]:
# 3. Se optimiza el modelo
study_optuna = optuna.create_study(direction = 'minimize', sampler = TPESampler(seed = 123))
study_optuna.optimize(objective, timeout=time_train)

In [50]:
# 3. Reportar el número de trials, el MAE y los mejores hiperparámetros encontrados.
best_param_optuna = study_optuna.best_params
mae_optuna = study_optuna.best_value
print(f'Número de trials: {len(study_optuna.trials)}')
print(f'Mejor MAE: {study_optuna.best_value}')
print(f'Mejores hiperparámetros: {study_optuna.best_params}')

Número de trials: 144
Mejor MAE: 1930.8952094729837
Mejores hiperparámetros: {'min_frequency': 0.02932190820903273, 'learning_rate': 0.08795069633994801, 'n_estimators': 710, 'max_depth': 10, 'max_leaves': 98, 'min_child_weight': 5, 'reg_alpha': 0.08275668675275855, 'reg_lambda': 0.11302033922936008}


4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?

* learning_rate: Corresponde a la tasa de aprendizaje del modelo, es decir que muestra que tan rápido se descendiendo por el gradiente hasta el valor óptimo. El rango dado es adecuado, ya que no usa valores mayores a 1, y tampoco valores muy pequeños. Además el desplazamiento que se realiza es lento, lo cual mejora el aprendizaje de los modelos.

* n_estimators: Corresponde al número de árboles de decisión que se construyen. Un valor muy alto puede conducir a un problema de sobreajuste, ya que un número muy grande de estos puede causar perdida de generalidad. El rango dado tambien es adecuado.

* max_depth: Corresponde a la profundidad máxima que tiene cada árbol. Nuevamente, su rango es adecuado ya que no deberia llegar a sobreajustarse el modelo.

* max_leaves: Corresponde al número máximo de nodos que posee cada árbol. Al igual que el caso anterior, el rango es razonable para este tipo de tareas.

* min_child_weight: Es el peso mínimo que se necesita para crear un nuevo nodo en los árboles. Si su valor es muy alto, se evita sobreajustar, por lo que el rango es adecuado.

* rag_alpha: Regularización L1 aplicada a los pesos de las caracteristicas, este ayuda a prevenir el sobreajuste al penalizar los coeficientes que tienen las caracteristicas. Nuevamente, su rango es adecuado.

* rag_lambda: Regularización L2 aplicada a los pesos de las caracteristicas, este penaliza los cuadrados de los coeficientes de las características. Su rango es aceptable.

In [51]:
# 5. Guardar su modelo en un archivo .pkl
import pickle

# Obtenemos mejor pipeline
pipeline_optuna = study_optuna.best_trial.user_attrs['pipeline_optuna']

with open('pipeline_optuna.pkl', 'wb') as f:
    pickle.dump(pipeline_optuna, f)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [52]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.4 MB/s eta 0:00:00


### ¿Qué es prunning?
El pruning es una técnica que permite reducir el tamaño y la complejidad de un modelo perdiendo la menor cantidad posible de rendimiento. Funciona deteniendo de manera anticipada aquellas configuraciones que no muestran mejoras prometedoras durante el proceso de entrenamiento. Esto puede afectar la velocidad del entrenamiento ya que, al contar con menos parámetros, se debería poder entrenar el modelo de manera más rápida.

In [53]:
import optuna
from optuna.integration import XGBoostPruningCallback
import xgboost as xgb
from xgboost import XGBRegressor

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [56]:
# Se redefine la función objective
def objective(trial):
    # Se determina frecuencia mínima para OneHotEncoder
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Generamos diccionario con variables a optimizar
    params_xgb = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'eval_metric': 'mae'
    }

    # Generamos col transformer
    col_transformer = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_data),
            ('categorical', OneHotEncoder(min_frequency=min_frequency, sparse_output=False), categorical_data)
        ])
    col_transformer.set_output(transform='pandas')

    # Transformamos los datos y se preparan para XGBoost
    X_train_transformed = col_transformer.fit_transform(X_train)
    X_val_transformed = col_transformer.transform(X_val)
    dtrain = xgb.DMatrix(X_train_transformed, label=y_train)
    dval = xgb.DMatrix(X_val_transformed, label=y_val)

    # Agregamos el callback de pruning
    pruning_callback = XGBoostPruningCallback(trial, 'validation-mae')

    # Se define el modelo con los parámetros seleccionados por Optuna
    evals = [(dtrain, 'train'), (dval, 'validation')]
    model = xgb.train(
        params_xgb,
        dtrain,
        num_boost_round=params_xgb['n_estimators'],
        early_stopping_rounds=10,
        evals=evals,
        callbacks=[pruning_callback],
        verbose_eval=False
    )

    # Predecir y calcular MAE
    val_predictions = model.predict(dval)
    mae = mean_absolute_error(y_val, val_predictions)

    # Almacenamos mejor pipeline
    trial.set_user_attr("pipeline_prunning", model)

    return mae_xgb

In [57]:
# Se configura el estudio de optuna
study = optuna.create_study(direction="minimize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, timeout=300, show_progress_bar=True)

   0%|          | 00:00/05:00

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:29:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:29:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:29:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:29:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [01:29:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.



In [58]:
# Se obtienen los mejores parámetros y su MAE
best_param_pruning = study.best_params
mae_pruning = study.best_value

print(f"Mejor MAE: {study.best_value}")
print(f"Mejores hiperparámetros: {study.best_params}")
print(f"Número de trials: {len(study.trials)}")

Mejor MAE: 2433.320936196607
Mejores hiperparámetros: {'min_frequency': 0.3745401188473625, 'learning_rate': 0.0951207163345817, 'n_estimators': 746, 'max_depth': 7, 'max_leaves': 15, 'min_child_weight': 1, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352}
Número de trials: 505


### ¿Cómo cambian los resultados? ¿A que se peude deber esto?

Es posible observar que la optimización con pruning muestra un mejor rendimiento del modelo, alcanzando un MAE de 6477. Esta mejora se debe a que, al detener el procedo de entrenamiento de modelos que no muestran mejoría, el modelo puede enfocarse en iteraciones más efectivas, optimizando así su capacidad de predicción.

In [59]:
# Obtenemos mejor pipeline
pipeline_prunning = study.best_trial.user_attrs['pipeline_prunning']

with open('pipeline_prunning.pkl', 'wb') as f:
    pickle.dump(pipeline_prunning, f)

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

# 1. Grafico de historial de optimizacion
plot_optimization_history(study)

In [ ]:
# 2. Gráfico de coordenadas paralelas
plot_parallel_coordinate(study)

In [ ]:
# 3. Gráfico de importancia de hiperparámetros
plot_param_importances(study)

4. ¿Desde qué trial se empiezan a observar mejoras notables en sus resultados?

El historial de optimización muestra que las mejoras ocurren rápidamente, alcanzando los valores objetivos alrededor del quinto o décimo trial. La curva generada por los trials indica que, en el primer trial, los valores son considerablemente altos, pero se estabilizan de forma rápida a medida que avanza la optimización.


5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?

El gráfico muestra las tendencias de diversos hiperparámetros durante el proceso de optimización. Algunas variables, como learning_rate y max_depth, tienen poca variabilidad en sus valores, lo cual se refleja en la escasez de líneas paralelas. En cambio, otras variables como max_leaves, min_frequency, reg_alpha y reg_lambda presentan una alta variabilidad, evidenciada por la abundancia de líneas con diferentes valores.

6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

Del gráfico de importancia de los hiperparámetros, se observa que min_frequency es significativamente más relevante, con una importancia de 0.73. Los siguientes en importancia, reg_lambda y n_estimators, tienen un valor mucho menor, descendiendo a 0.08. Esto sugiere que el resto de los hiperparámetros tienen una influencia limitada en el rendimiento del modelo.


## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [60]:
# Se crea la data que tendrá la tabla
mae_resume = {
    'Model': ['Modelo dummy', 'Model XGBRegressor', 'Model XGBRegressor Monotone', 'Model Optuna', 'Model Pruning'],
    'MAE': [mae_dummy, mae_xgb, mae_xgb_monotone, mae_optuna, mae_pruning]
}

# Crear tabla en DataFrame
mae_results = pd.DataFrame(mae_resume)
mae_results

,Model,MAE
0,Modelo dummy,13298.497767
1,Model XGBRegressor,2433.320936
2,Model XGBRegressor Monotone,2485.266687
3,Model Optuna,1930.895209
4,Model Pruning,2433.320936


### ¿Qué modelo obtiene el mejor rendimiento?
Es posible observar que el modelo que presenta un mejor rendimiento es el modelo optuna, pues presenta un MAE de 6477, siendo el que presenta un menor error en las predicciones.



In [63]:
# Se carga el mejor modelo

with open('pipeline_optuna.pkl', 'rb') as f:
    best_model = pickle.load(f)

# Se predice sobre el conjunto test
predictions = best_model.predict(X_test)

# Se calcula el MAE
mae_test = mean_absolute_error(y_test, predictions)
print(f'El MAE del modelo sobre el conjunto de test es: {mae_test}')

El MAE del modelo sobre el conjunto de test es: 1902.898215286214


### ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?
Podemos observar que no existe tanta diferencia con respecto a las métricas obtenidas anteriormente, pues ahora se obtuvo un valor similar. Lo cual tiene sentido considerando la naturaleza de los datos, ya que existe una consistencia entre las predicciones realizadas y no ocurrio un sobreajuste en los datos de validación. Por lo que el modelo, fue capaz de capturar patrones que no se repiten en los datos de test.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>